# ふらっとPythonの会 

#### 2022 / 3 / 7

## 回帰分析

コンピュータを用いて、統計分析に挑戦します。

統計分析を行う目的は、データの理解、状態（<b>変化</b>）の把握、自分の主張のエビデンスの作成、などさまざまです。

分析は、より客観的に、より説得力の高いものを目指そうとすると、統計理論、数学的理論に沿った多くのステップを踏まなければなりません。

今回は、そこまでのものは目指さず、データの理解、Pythonに慣れることを目的とした、簡易的な分析を行うこととします。

作業には、以下のステップを踏みます。

- インターネットから（成形済み）データのダウンロード
- データの可視化
- 回帰分析
- 結果の確認

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 3_1. データのダウンロード

`02_python_analysis.ipynb` でもやりました、インターネットからデータのダウンロードを行います。

URLは変えなければなりませんが、ほとんどのコードを流用することができます。

今回は、3つのファイルをダウンロードします。

In [ ]:
import requests

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/'

filename='winequality-red.csv'
urlData = requests.get(url + filename).content

with open(filename, mode='wb') as f:
    f.write(urlData)

filename='winequality-white.csv'
urlData = requests.get(url + filename).content
with open(filename, mode='wb') as f:
    f.write(urlData)
    
filename='winequality.names'
urlData = requests.get(url + filename).content
with open(filename ,mode='wb') as f:
    f.write(urlData)

#### 参考

今回利用するWebサイト（ http://archive.ics.uci.edu/ml/machine-learning-databases/ ）は、データ分析の初学者や、Pythonの練習者に対して公開されている、練習用データです。

（練習といっても、中身は本物です。）

今回使用するデータ以外にも多様なデータがあるので、ぜひ利用してみてください。

今回は、3つのファイルをダウンロードします。

ダウンロードしたファイルのタイトルから推測できる通り、今回は、ワインの品質検査の結果をまとめたデータを使い、ワインの成分と品質の関係について分析します。

- `winequality-red.csv`: 赤ワインの分析データと品質
- `winequality-white.csv`: 白ワインの分析データと品質
- `winequality.names`: データの説明

以上のファイルがダウンロードできたら、 `winequality.names` を読んでみてください（Jupyter notebook上ではファイルをクリックするだけで見ることができるはずです）。



In [ ]:
# 保存したデータを読み込みます。
red_wine = pd.read_csv('winequality-red.csv',sep=';')
white_wine = pd.read_csv('winequality-white.csv',sep=';')

In [ ]:
# 赤ワインのデータの最初の5つ
red_wine.head()

In [ ]:
# 白ワインのデータの最初の5つ
white_wine.head()

`winequality.names`に詳しい説明がありますが、それぞれの変数について簡単にまとめておきます。

|No.|変数名|日本語訳|単位（記号）|
|-|-|-|-|
|1|fixed acidity|固定酸性度（酒石酸）|$g/dm^3$|
|2|volatile acidity|揮発性酸性度（酢酸）|$g/dm^3$|
|3|citric acid|クエン酸|$g/dm^3$|
|4|residual sugar|残留糖|$g/dm^3$|
|5|chlorides|塩化ナトリウム濃度|$g/dm^3$|
|6|free sulfur dioxide|遊離二酸化硫黄（亜硫酸）|$mg/dm^3$|
|7|total sulfur dioxide|総二酸化硫黄（亜硫酸）|$mg/dm^3$|
|8|density|比重|$g/dm^3$|
|9|pH|pH|-|
|10|sulphates|硫酸塩（硫酸カリウム）|$g/dm^3$|
|11|alcohol|アルコール濃度|$ \%$|
|12|quality|品質検査結果|-|

ワインに詳しくない人も、化学に明るくない人も、一度読んでみましょう。

統計分析や機械学習は、パソコン上でおこなう場合は、変数の意味を深く理解せずとも機械的に進めることもできます。

コンピュータの性能の向上によって、数百万件規模のデータを総当たり的に比較したり、何通りもの分析手法を並行で行うことができるようになりました。

これにより、統計学の知識が不十分でも、やろうと思えば、専門家に匹敵する正確なモデルを導出することも不可能ではなくなりました。しかし、意味を理解してから分析を行うことは、効率よく分析を進めることができるほか、見識を広げる上でも役に立ちます。

例え苦手な分野だとしても、データを理解するためのプロセスは必ず踏みましょう。

## 3_2. データの理解

新しくデータを取得した際、どのような特徴を持つデータなのか、データの理解する作業は必要です。

ひと目で確認することが困難なほどデータ数が多い場合には、「集計する」、「図示する」などが有効な手段です。

`DataFrame`型には、データのサマリーを出力する便利な関数があります。

- pandas.DataFrame.info()

`DataFrame`型データの各列のデータ型や、異常値がないかを確認することができます。

In [ ]:
# 赤ワインデータの全体像を見る
red_wine.info()

- pandas.DataFrame.describe()

`DataFrame`型データの各列のデータの平均値や標準偏差（分散）等を確認することができます。

In [ ]:
# 赤ワインデータの各列を確認する
red_wine.describe()

### Ex. 3_1

1. それぞれの特徴量（各列のことを特徴量といいます）がどのようなデータになっているか考えてみてください。
2. 赤ワインと白ワインのデータを比較してみましょう。何がわかりますか？

### （参考）箱ひげ図

箱ひげ図とは、データの散らばりを単純な図形で表現したものです。

中央値、第一四分位、第三四分位、最小値、最大値、外れ値がわかります。


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8,6))
plt.boxplot(red_wine["alcohol"],
            labels=["alcohol"])

plt.grid(axis="y")
plt.show()

In [ ]:
#赤ワインのアルコールと白ワインのアルコールを比較してみる

plt.figure(figsize=(8,6))
plt.boxplot([red_wine["alcohol"], white_wine["alcohol"]],
            labels=["alcohol_red", "alcohol_white"])

plt.grid(axis="y")
plt.show()

分析を行う上で一番気になるのはやはり品質の結果（`quality`）です。

`quality`は、カテゴリ変数（0 - 10 までの整数データ）なので、棒グラフで表現します。

In [ ]:
#赤ワインのアルコールと白ワインのqualityを比較してみる

red_wine_quality = red_wine.groupby("quality")["quality"].count()
plt.bar(x = red_wine_quality.index, height = red_wine_quality)
plt.grid(axis="both")
plt.show()

In [ ]:
white_wine_quality = white_wine.groupby("quality")["quality"].count()
plt.bar(x = white_wine_quality.index, height = white_wine_quality)
plt.grid(axis="both")
plt.show()

### 相関関係と散布図

データの特徴を理解する上で、最も重要な項目の一つが相関関係です。

2つのデータが相互に影響している可能性がある場合、相関関係に表れます。

例えば、`quality` と `alcohol` に相関関係はないかどうか、調べてみます。

相関関係は、散布図を作成すれば、すぐにわかります。

In [ ]:
# quality と alcohol の関係を散布図で示す（赤ワイン）
plt.scatter(x=red_wine['quality'], y=red_wine['alcohol'])
plt.show()

In [ ]:
# quality と alcohol の関係を散布図で示す（白ワイン）
plt.scatter(x=white_wine['quality'], y=white_wine['alcohol'])
plt.show()

いかがでしょうか。

相関関係があるような、ないような、判断が少し難しいかもしれません。

相関関係は、相関係数という数値で決まりますが、相関係数は計算で出すことができます。

In [ ]:
# quality と alcohol の相関係数（赤ワイン）
np.corrcoef(red_wine["quality"], red_wine["alcohol"])

In [ ]:
# quality と alcohol の相関係数（白ワイン）
np.corrcoef(white_wine["quality"], white_wine["alcohol"])

相関係数の目安はおおむね以下のとおりです。

- 0.6 - 0.8 強い相関関係
- 0.4 - 0.6 弱い相関関係
- 0.2 - 0.4 相関関係はない

### Ex. 3_2

1. 赤ワインと白ワインについて、品質（`quality`）と、残留糖（`residual sugar`）の散布図を作成し、相関係数を計算してみましょう。

In [ ]:
# quality と residual sugar の関係を散布図で示す（赤ワイン）
plt.scatter(x=red_wine['quality'], y=red_wine['residual sugar'])
plt.show()

In [ ]:
# quality と residual sugar の関係を散布図で示す（白ワイン）
plt.scatter(x=white_wine['quality'], y=white_wine['residual sugar'])
plt.show()

In [ ]:
# quality と residual sugar の相関係数（赤ワイン）
np.corrcoef(red_wine["*******"], red_wine["*******"])

In [ ]:
# quality と residual sugar の相関係数（白ワイン）
np.corrcoef(white_wine["*******"], white_wine["*******"])

### Ex. 3_3

1. 赤ワインと白ワインについて、アルコール度数（`alcohol`）と、残留糖（`residual sugar`）の散布図を作成し、相関係数を計算してみましょう。そこから何かわかることはあるでしょうか。

In [ ]:
# residual sugar と alcohol の関係を散布図で示す（赤ワイン）
plt.scatter(x=red_wine['alcohol'], y=red_wine['residual sugar'])
plt.title("alcohol vs. residual sugar - red wine")
plt.xlabel("alcohol")
plt.ylabel("residual sugar")
plt.show()

In [ ]:
# residual sugar と alcohol の関係を散布図で示す（白ワイン）
plt.scatter(x=white_wine['alcohol'], y=white_wine['residual sugar'])
plt.title("alcohol vs. residual sugar - white wine")
plt.xlabel("alcohol")
plt.ylabel("residual sugar")
plt.show()

In [ ]:
# alcohol と residual sugar の相関係数（赤ワイン）
np.corrcoef(red_wine["*******"], red_wine["*******"])

In [ ]:
# alcohol と residual sugar の相関係数（白ワイン）
np.corrcoef(white_wine["*******"], white_wine["*******"])

これまで、個々の特徴量について比較してきました。

最後に、全ての特徴量の相関係数を出して、図示してみます。

In [ ]:
# seaborn というライブラリを新しくインストールします。
!pip install seaborn

In [ ]:
# seaborn で相関係数をヒートマップ形式で図示します。
import seaborn as sns

corr = red_wine.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, vmax=1, vmin=-1, center=0, annot=True, cbar=True, cmap='coolwarm', square=True, fmt='.2f', ax=ax)

quality と比較的強い相関を持つのは、 「alcohol」、「volatile acidity」、「sulphates」、「critic acid」ということがわかりました。

## 3_3. 最大値と最小値

今回のデータでは、特徴量から品質（`quality`）の傾向を分析しますので、

データを品質（`quality`）ごとに集計し、それぞれの特徴量の最大値と最小値を計算しましょう。

`02_python_analysis.ipynb` で、データを集計する前処理を行いましたが、今回も同様の関数を用います。

In [ ]:
wine_min_max = white_wine.groupby('quality')

In [ ]:
#qualityごとの最大値を表示します。
wine_min_max.max()

In [ ]:
#qualityごとの最大値を表示します。
wine_min_max.min()

In [ ]:
#qualityごとの平均値を表示します。
wine_min_max.mean()

In [ ]:
#qualityごとの平均値を表示します。
wine_min_max.median()

最大値、最小値の比較には、箱ひげ図が便利です。

In [ ]:
# 赤ワイン
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12,6))
plt.boxplot([red_wine[red_wine["quality"] == 3]["alcohol"],
            red_wine[red_wine["quality"] == 4]["alcohol"],
            red_wine[red_wine["quality"] == 5]["alcohol"],
            red_wine[red_wine["quality"] == 6]["alcohol"],
            red_wine[red_wine["quality"] == 7]["alcohol"],
            red_wine[red_wine["quality"] == 8]["alcohol"]],
            labels=["alcohol: quality = 3",
                    "alcohol: quality = 4",
                    "alcohol: quality = 5",
                    "alcohol: quality = 6",
                    "alcohol: quality = 7",
                    "alcohol: quality = 8"
                   ])

plt.grid(axis="y")
plt.show()

## 3_4.重回帰分析

最後に、ワインの成分データから `quality` を求めるモデルを作成します。

統計学においてモデルとは、たいてい「数理モデル」のことを指します。

In [ ]:
from sklearn.model_selection import train_test_split

X = red_wine.drop('quality', axis=1)
y = red_wine['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size =0.8, random_state=0)

In [ ]:
import statsmodels.api as sm

# 回帰モデルを作成します(OLSとは、Ordinary Least Square: 最小二乗法のことです)。
mod = sm.OLS(y_train, sm.add_constant(X_train))

# モデルを使って学習します。
result = mod.fit() 

# 統計サマリを表示
print(result.summary())

### （参考）

- R-squared: 決定係数。被説明変数の動きのうち、説明変数の動きで説明できる割合=回帰式の当て はまりの良さを示す。1が最も良い。

- F-statistic: F値。回帰式が意味があるかどうかを検定する統計量。

- Prob (F-statistic): (F 検定に基づく)p 値。回帰式が意味が無い(全ての説明変数の係数がゼロである)確率。

- t: t値。係数の有意性(意味がある説明変数かどうか)を検定するための統計量。 t 値=係数の推定値/係数の標準誤差。概ね 2 より大きければ良い。

- P>|t|: p 値(t 検定に基づく)。説明変数として意味の無い(係数がゼロである)確率。小さければ意味のある説明変数である(「有意」である)と判断。